In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
sns.set_style('darkgrid')
from wordcloud import WordCloud,STOPWORDS
import re
stopwords = list(STOPWORDS)
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import PolynomialFeatures,StandardScaler
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

plt.rc('figure',figsize=(20,11))

<h3 style="background-color:skyblue;font-family:newtimeroman;font-size:300%;text-align:center">Introduction</h3>

![](https://www.bornontario.ca/en/about-born/resources/Images/books-1204029_1920.jpg)



<p style="text-align: center;"><br></p>
<p style="text-align: center;"><span style="font-size: 24px;"><span style="font-family: 'Times New Roman', Times, serif;"><br></span></span></p>
<p style="text-align: center;"><span style="font-size: 24px;"><span style="font-family: 'Times New Roman', Times, serif;">There are many features that make up a good book, but what are the features which affect most the goodness of a programming or computer science book?</span></span></p>
<p style="text-align: center;"><span style="font-size: 24px;"><span style="font-family: 'Times New Roman', Times, serif;">In this kernel, we will dive into 270 examples of books and try to understand what makes a book score high review scores, what affects the book&apos;s price, and are there any underlying patterns we are missing, would unraveling them help book writers make better books?</span></span></p>
<p style="text-align: center;"><span style="font-size: 24px;"><span style="font-family: 'Times New Roman', Times, serif;"><br></span></span></p>
<h3 style="text-align: center;"><span style="font-size: 24px;"><span style="font-family: 'Times New Roman', Times, serif;">Questions We Will Try To Answer</span></span></h3>
<p style="text-align: center;"><span style="font-size: 24px;"><span style="font-family: 'Times New Roman', Times, serif;"><br></span></span></p>
<p style="text-align: center;"><span style="font-size: 24px;"><span style="font-family: 'Times New Roman', Times, serif;">1) What features are correlated with the price features and the score feature</span></span></p>
<p style="text-align: center;"><span style="font-size: 24px;"><span style="font-family: 'Times New Roman', Times, serif;"><br></span></span></p>
<p style="text-align: center;"><span style="font-size: 24px;"><span style="font-family: 'Times New Roman', Times, serif;">2) How the number of pages affect the rest of our features.</span></span></p>
<p style="text-align: center;"><span style="font-size: 24px;"><span style="font-family: 'Times New Roman', Times, serif;"><br></span></span></p>
<p style="text-align: center;"><span style="font-size: 24px;"><span style="font-family: 'Times New Roman', Times, serif;">3) Are any programming languages come up more frequently?</span></span></p>
<p style="text-align: center;"><span style="font-size: 24px;"><span style="font-family: 'Times New Roman', Times, serif;"><br></span></span></p>
<p style="text-align: center;"><span style="font-size: 24px;"><span style="font-family: 'Times New Roman', Times, serif;">4) Can we divide the books by categories and understand what category costs more or has higher review scores.</span></span></p>
<p style="text-align: center;"><span style="font-size: 24px;"><span style="font-family: 'Times New Roman', Times, serif;"><br></span></span></p>
<p style="text-align: center;"><span style="font-size: 24px;"><span style="font-family: 'Times New Roman', Times, serif;"><br></span></span></p>
<h3 style="text-align: center;"><span style="font-size: 24px;"><span style="font-family: 'Times New Roman', Times, serif;">&nbsp;Feature Engineering Goals</span></span></h3>
<p style="text-align: center;"><span style="font-size: 24px;"><span style="font-family: 'Times New Roman', Times, serif;">1) Extract description text features like text length, average word length, programming language, etc.</span></span></p>
<p style="text-align: center;"><span style="font-size: 24px;"><span style="font-family: 'Times New Roman', Times, serif;"><br></span></span></p>
<p style="text-align: center;"><span style="font-size: 24px;"><span style="font-family: 'Times New Roman', Times, serif;">2) Extract book title text features like text length, average word length, programming language, etc.</span></span></p>
<p style="text-align: center;"><span style="font-size: 24px;"><span style="font-family: 'Times New Roman', Times, serif;"><br></span></span></p>
<h3 style="text-align: center;"><span style="font-size: 24px;"><span style="font-family: 'Times New Roman', Times, serif;">Prediction Goals</span></span></h3>
<p style="text-align: center;"><span style="font-size: 24px;"><span style="font-family: 'Times New Roman', Times, serif;">1) Predict the number of potential reviews for a book &nbsp;</span></span></p>
<p style="text-align: center;"><span style="font-size: 24px;"><span style="font-family: 'Times New Roman', Times, serif;"><br></span></span></p>
<p style="text-align: center;"><span style="font-size: 24px;"><span style="font-family: 'Times New Roman', Times, serif;">2) Predict a book&apos;s rating&nbsp;</span></span></p>
<p style="text-align: center;"><span style="font-size: 24px;"><span style="font-family: 'Times New Roman', Times, serif;"><br></span></span></p>
<p style="text-align: center;"><span style='font-family: "Times New Roman", Times, serif; font-size: 24px;'>3) Predict a book&apos;s price</span></p>


In [ ]:
b_data = pd.read_csv('/kaggle/input/top-270-rated-computer-science-programing-books/prog_book.csv')
b_data.head(3)

<h3 style="background-color:skyblue;font-family:newtimeroman;font-size:250%;text-align:center">Feature Engineering</h3>


In [ ]:
b_data['Title_Average_Word_Length'] = b_data.Book_title.apply(lambda x: np.mean([len(word) for word in x.split()]))
b_data['Title_Number_Of_Words'] = b_data.Book_title.apply(lambda x: len(x.split()))
b_data['Description_Average_Word_Length'] = b_data.Description.apply(lambda x: np.mean([len(word) for word in x.split()]))
b_data['Description_Number_Of_Words'] = b_data.Description.apply(lambda x: len(x.split()))


lencoder = LabelEncoder()
lencoder.fit(b_data.Type)
b_data.Type = lencoder.transform(b_data.Type)
b_data.Reviews = b_data.Reviews.apply(lambda x : int(x.replace(',','')))

<h3 style="background-color:skyblue;font-family:newtimeroman;font-size:250%;text-align:center">Exploratory data analysis</h3>


In [ ]:
plt.subplot(3,1,1)
ax = sns.boxplot(b_data.Rating,color='tab:green',showmeans=True,meanprops={"marker":"+",
                       "markerfacecolor":"white", 
                       "markeredgecolor":"black",
                      "markersize":"10"})
ax.set_title('Distribution Of Rating Scores Across Our Dataset',fontsize=21)

plt.subplot(3,1,2)
ax = sns.distplot(b_data.Rating,label="Ratings",color='green')
ax.set_xlabel("Rating",fontsize=20)
ax.set_ylabel("Density",fontsize=20)
textstr = '\n'.join(
        (r'$\mu=%.2f$' % (b_data.Rating.mean(),), r'$\mathrm{median}=%.2f$' % (b_data.Rating.median(),),
         r'$\sigma=%.2f$' % (b_data.Rating.std(),)))
props = dict(boxstyle='round', facecolor='green', alpha=0.5)
ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=14,
            verticalalignment='top', bbox=props)


ax.axvline(b_data['Rating'].mean(), color='r', linestyle='--')
ax.axvline(b_data['Rating'].median(), color='tab:orange', linestyle='-')
ax.legend(['Mean','Median'])
plt.subplot(3,1,3)
ax = sns.kdeplot(b_data.Rating,label="Ratings",color='green',cumulative = True)
ax.set_xlabel("Rating",fontsize=20)
ax.set_ylabel("Density",fontsize=20)

plt.show()

<p style="text-align: center;"><span style='font-family: "Times New Roman", Times, serif; font-size: 24px;'>Interesting! We can see that our ratings are normally distributed around 4.0, unlike some datasets where the ratings are usually skewed towards low scores or high scores our sample population is normally distributed which is good news for our future modeling.</span></p>

In [ ]:
plt.subplot(2,1,1)
ax = sns.distplot(b_data.Reviews,label="Reviews",color='teal')
ax.set_xlabel("Reviews",fontsize=20)
ax.set_ylabel("Density",fontsize=20)
textstr = '\n'.join(
        (r'$\mu=%.2f$' % (b_data.Reviews.mean(),), r'$\mathrm{median}=%.2f$' % (b_data.Reviews.median(),),
         r'$\sigma=%.2f$' % (b_data.Reviews.std(),)))
props = dict(boxstyle='round', facecolor='teal', alpha=0.5)
ax.text(0.85, 0.85, textstr, transform=ax.transAxes, fontsize=14,
            verticalalignment='center', bbox=props)

ax.set_title('Distribution Of Reviews Across Our Dataset',fontsize=21)
plt.show()


b_data.Reviews = b_data.Reviews.replace(0,1)
b_data.Reviews = np.log(b_data.Reviews)
plt.subplot(2,1,2)
ax = sns.distplot(b_data.Reviews,label="Reviews",color='teal')
ax.set_xlabel("Reviews",fontsize=20)
ax.set_ylabel("Density",fontsize=20)
textstr = '\n'.join(
        (r'$\mu=%.2f$' % (b_data.Reviews.mean(),), r'$\mathrm{median}=%.2f$' % (b_data.Reviews.median(),),
         r'$\sigma=%.2f$' % (b_data.Reviews.std(),)))
props = dict(boxstyle='round', facecolor='teal', alpha=0.5)
ax.text(0.85, 0.85, textstr, transform=ax.transAxes, fontsize=14,
            verticalalignment='center', bbox=props)

ax.set_title('Distribution Of Reviews Across Our Dataset After Log Transformation',fontsize=21)
plt.show()




<p style="text-align: center;"><span style='font-family: "Times New Roman", Times, serif; font-size: 24px;'>Unlike our ratings, the number of reviews is distributed almost exponentially, we can clearly see that the massive skews is caused by outliers, we will perform a logarithmic transformation of this feature in order to rebalance our data in comparisons to outliers.
After performing a logarithmic transformation we are left with a fairly normal distribution where our mean and median are almost the same as well as a fairly small standard deviation, although we can see two picks which suggest that although the average logarithmic scale number of reviews is 3.3 a large portion of the books have 0 reviews..</span></p>

In [ ]:
plt.subplot(2,1,1)
ax = sns.distplot(b_data.Number_Of_Pages,label="Number_Of_Pages",color='red')
ax.set_xlabel("Number Of Pages",fontsize=20)
ax.set_ylabel("Density",fontsize=20)
textstr = '\n'.join(
        (r'$\mu=%.2f$' % (b_data.Number_Of_Pages.mean(),), r'$\mathrm{median}=%.2f$' % (b_data.Number_Of_Pages.median(),),
         r'$\sigma=%.2f$' % (b_data.Number_Of_Pages.std(),)))
props = dict(boxstyle='round', facecolor='red', alpha=0.5)
ax.text(0.85, 0.85, textstr, transform=ax.transAxes, fontsize=14,
            verticalalignment='center', bbox=props)

ax.set_title('Distribution Of Book Page Number Counts',fontsize=21)
plt.show()

b_data = b_data[b_data['Number_Of_Pages']<1500]

plt.subplot(2,1,2)
ax = sns.distplot(b_data.Number_Of_Pages,label="Number_Of_Pages",color='red')
ax.set_xlabel("Number Of Pages",fontsize=20)
ax.set_ylabel("Density",fontsize=20)
textstr = '\n'.join(
        (r'$\mu=%.2f$' % (b_data.Number_Of_Pages.mean(),), r'$\mathrm{median}=%.2f$' % (b_data.Number_Of_Pages.median(),),
         r'$\sigma=%.2f$' % (b_data.Number_Of_Pages.std(),)))
props = dict(boxstyle='round', facecolor='red', alpha=0.5)
ax.text(0.85, 0.85, textstr, transform=ax.transAxes, fontsize=14,
            verticalalignment='center', bbox=props)

ax.set_title('Distribution Of Book Page Number Counts',fontsize=21)
plt.show()

In [ ]:
ax = sns.countplot(b_data.Type)
ax.set_xticklabels(lencoder.inverse_transform([0,1,2,3,4,5]))
ax.set_xlabel("Book Type",fontsize=20)
ax.set_ylabel("Count",fontsize=20)
ax.set_title('Distibution Of Different Book Types In Our Data',fontsize=22)


<p style="text-align: center;"><span style='font-family: "Times New Roman", Times, serif; font-size: 24px;'>Most of the books in our dataset are Paperback which is somewhat surprising taken in mind that we have so many alternatives for reading books using our phones, etc but still most of the books in our data are paperback. This may also suggest that most of these books are of older releases.</span></p>

In [ ]:
plt.subplot(2,1,1)
ax = sns.distplot(b_data.Price,label="Price")
ax.set_xlabel("Price",fontsize=20)
ax.set_ylabel("Density",fontsize=20)
textstr = '\n'.join(
        (r'$\mu=%.2f$' % (b_data.Price.mean(),), r'$\mathrm{median}=%.2f$' % (b_data.Price.median(),),
         r'$\sigma=%.2f$' % (b_data.Price.std(),)))
props = dict(boxstyle='round', facecolor='red', alpha=0.5)
ax.text(0.85, 0.85, textstr, transform=ax.transAxes, fontsize=14,
            verticalalignment='center', bbox=props)

ax.set_title('Distribution Of Book Prices',fontsize=21)
plt.show()
b_data.Price = np.log(b_data.Price)
plt.subplot(2,1,2)
ax = sns.distplot(b_data.Price,label="Price")
ax.set_xlabel("Price",fontsize=20)
ax.set_ylabel("Density",fontsize=20)
textstr = '\n'.join(
        (r'$\mu=%.2f$' % (b_data.Price.mean(),), r'$\mathrm{median}=%.2f$' % (b_data.Price.median(),),
         r'$\sigma=%.2f$' % (b_data.Price.std(),)))
props = dict(boxstyle='round', facecolor='red', alpha=0.5)
ax.text(0.85, 0.85, textstr, transform=ax.transAxes, fontsize=14,
            verticalalignment='center', bbox=props)

ax.set_title('Distribution Of Book Prices',fontsize=21)
plt.show()




<p style="text-align: center;"><span style='font-family: "Times New Roman", Times, serif; font-size: 24px;'>As with the other features we have a few outliers in the price range of our books but if we ignore the outliers we can see our distribution is centered around 46 USD and the distribution curve is quite narrow meaning that our standard deviation around the mean price is fairly low when ignoring the outliers.</span></p>

In [ ]:

title_w = ''

for word in b_data.Book_title:
    word = word.lower()
    splited = re.findall(r'\b[A-Za-z]+\b',word)
    splited = [w for w in splited if w not in stopwords]
    title_w += ' '.join(splited)+ ' '


wordcloud = WordCloud(width = 1100, height = 800, 
                background_color ='white', 
                stopwords = stopwords, 
                min_font_size = 8).generate(title_w) 
  
# plot the WordCloud image                        
plt.figure(figsize = (18, 11), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
  
plt.show() 

<p style="text-align: center;"><span style='font-family: "Times New Roman", Times, serif; font-size: 24px;'>Not very surprising but we can see that the most common words in our book titles are programing related together with words like Introduction and Guide.
</span></p>

In [ ]:
desc_w = ''

for word in b_data.Description:
    word = word.lower()
    splited = re.findall(r'\b[A-Za-z]+\b',word)
    splited = [w for w in splited if w not in stopwords]
    desc_w += ' '.join(splited)+ ' '


wordcloud = WordCloud(width = 1100, height = 800, 
                background_color ='white', 
                stopwords = stopwords, 
                min_font_size = 8).generate(desc_w) 
  
# plot the WordCloud image                        
plt.figure(figsize = (18, 11), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
  
plt.show() 



<p style="text-align: center;"><span style='font-family: "Times New Roman", Times, serif; font-size: 24px;'>The description of the book similarly to the title of the book has words that dominate most of the description such as 'programming' and 'language', 'design' and 'computer' which is not surprising although one would expect more references to typical programming language names and we can see that the names of the programming languages are not as visible as the other words stated already.
</span></p>

In [ ]:
plt.figure(figsize=(20,11))
ax = sns.jointplot(x=b_data.Price,y=b_data.Rating,height=10,kind='kde',cmap='mako')
#ax.set_xlabel("Price",fontsize=20)
#ax.set_ylabel("Density",fontsize=20)
#ax.set_title('Distribution Of Book Prices',fontsize=21)
plt.show()


<p style="text-align: center;"><span style='font-family: "Times New Roman", Times, serif; font-size: 24px;'>Most of the books in or data set are books in the price range 40-50 USD and rating range 4.0-4.5
</span></p>

In [ ]:
correlations = b_data.corr('pearson')
plt.figure(figsize=(20,11))
ax = sns.clustermap(correlations,annot=True,cmap='coolwarm',figsize=(20,11))


<p style="text-align: center;"><span style='font-family: "Times New Roman", Times, serif; font-size: 24px;'>We can see that we have some interesting correlations between our price and other features such as the number of reviews and the number of pages. we cannot use the number of reviews as a predictor although we have a fair correlation with the price feature it is a classic case of data leakage, we do not know the number of reviews until the book as published, but we can use the number of pages which has a high correlation and the types to estimate a book's price together with the book's type. as for the review scores, there is no significant correlation with any other feature but we will try using nearest neighbor models to approximate the review score.
</span></p>



<h3 style="background-color:skyblue;font-family:newtimeroman;font-size:250%;text-align:center">Model Selection And Evaluation</h3>


<h3 style="background-color:skyblue;font-family:newtimeroman;font-size:200%;text-align:center">Price Prediction</h3>


In [ ]:
features = ['Number_Of_Pages','Type','Title_Average_Word_Length']
train_x,test_x,train_y,test_y = train_test_split(b_data[features],b_data.Price)

LR_pipe = Pipeline(steps=[('model',LinearRegression())])
LR_scores = np.sqrt(-1*cross_val_score(LR_pipe,b_data[features],b_data.Price,cv=6,scoring='neg_mean_squared_error'))

plt.figure(figsize=(20,11))
ax = sns.pointplot(x=np.arange(0,6),y=LR_scores)
ax.set_title('Cross Validation RMSE for LinearRegression',fontsize=20)
ax.set_ylabel('RMSE',fontsize=18)
ax.set_xlabel('Fold Number',fontsize=18)

In [ ]:
print("LinearRegression Average Cross Validation Score:",LR_scores.mean())


<p style="text-align: center;"><span style='font-family: "Times New Roman", Times, serif; font-size: 24px;'>Can we lower our RMSE using a different model?
</span></p>



In [ ]:
RF_pipe = Pipeline(steps=[('model',RandomForestRegressor(n_estimators=50,max_leaf_nodes=15,random_state=42))])

RF_scores = np.sqrt(-1*cross_val_score(RF_pipe,b_data[features],b_data.Price,cv=6,scoring='neg_mean_squared_error'))

plt.figure(figsize=(20,11))
ax = sns.pointplot(x=np.arange(0,6),y=RF_scores,color='teal')
ax.set_title('Cross Validation RMSE for RandomForest',fontsize=20)
ax.set_ylabel('RMSE',fontsize=18)
ax.set_xlabel('Fold Number',fontsize=18)

In [ ]:
print("RandomForest Average Cross Validation Score:",RF_scores.mean())

In [ ]:
plt.figure(figsize=(20,11))
ax = sns.residplot(x=b_data.Number_Of_Pages,y=b_data.Price)
ax.set_title('Absolute residuals vs Price',fontsize=19,fontweight='bold')


<p style="text-align: center;"><span style='font-family: "Times New Roman", Times, serif; font-size: 24px;'>We can clearly see that our plot shows a heteroscedastic pattern meaning that regression will do us not good. We will still try polynomial regression for learning purposes.
</span></p>






<p style="text-align: center;"><span style='font-family: "Times New Roman", Times, serif; font-size: 24px;'>Not a significant change from Linear Regression
Lets plot and check out how our data is spread maybe a polynomial regression will improve are RMSE
</span></p>



In [ ]:
plt.figure(figsize=(20,11))
ax = sns.scatterplot(x=b_data.Number_Of_Pages,y=b_data.Price,hue=b_data.Type,palette='coolwarm')


<p style="text-align: center;"><span style='font-family: "Times New Roman", Times, serif; font-size: 24px;'>We see that the curve we want to fit is somewhat exponential, lets try and use Polynomial Regression with a polynomial of a 2 degree 
</span></p>



In [ ]:
pf = PolynomialFeatures(degree = 2)
PR_pipe = Pipeline(steps = [('scale',StandardScaler()),('pf',pf), ('model',LinearRegression()) ])



PR_scores = np.sqrt(-1*cross_val_score(PR_pipe,b_data[features],b_data.Price,cv=6,scoring='neg_mean_squared_error'))


plt.figure(figsize=(20,11))
ax = sns.pointplot(x=np.arange(0,6),y=PR_scores)
ax.set_title('Cross Validation RMSE for Polynomial Regression',fontsize=20)
ax.set_ylabel('RMSE',fontsize=18)
ax.set_xlabel('Fold Number',fontsize=18)

In [ ]:
print("Polynomial Regression Average Cross Validation Score:",PR_scores.mean())


<p style="text-align: center;"><span style='font-family: "Times New Roman", Times, serif; font-size: 24px;'>Polynomial Regression didn't improve our RMSE 
</span></p>



In [ ]:
KNR_pipe = Pipeline(steps=[('model',KNeighborsRegressor(n_neighbors=25))])

KNR_scores = np.sqrt(-1*cross_val_score(KNR_pipe,b_data[features],b_data.Price,cv=6,scoring='neg_mean_squared_error'))

plt.figure(figsize=(20,11))
ax = sns.pointplot(x=np.arange(0,6),y=KNR_scores,color='teal')
ax.set_title('Cross Validation RMSE for KNN',fontsize=20)
ax.set_ylabel('RMSE',fontsize=18)
ax.set_xlabel('Fold Number',fontsize=18)

In [ ]:
print("KNN Average Cross Validation Score:",KNR_scores.mean())


<p style="text-align: center;"><span style='font-family: "Times New Roman", Times, serif; font-size: 24px;'>So we tested a few models we saw that the best RMSE we got was using the good old linear regression, lets train and fit our model and check out will stacking our model result will improve the final RMSE score 
</span></p>



In [ ]:
b_data = b_data.sample(frac=1)
LR_pipe.fit(b_data[features],b_data.Price)
PR_pipe.fit(b_data[features],b_data.Price)
KNR_pipe.fit(b_data[features],b_data.Price)
RF_pipe.fit(b_data[features],b_data.Price)

In [ ]:
LR_Predict = LR_pipe.predict(b_data[features])

plt.figure(figsize=(20,11))
ax= sns.lineplot(x=np.arange(0,b_data.shape[0]),y=b_data.Price,label='Actual',color='green')
ax= sns.lineplot(x=np.arange(0,b_data.shape[0]),y=LR_Predict,label='LinearReg Prediciton',color='red')


In [ ]:
ens_Predict =LR_pipe.predict(b_data[features])*0.1+PR_pipe.predict(b_data[features])*0.3 + RF_pipe.predict(b_data[features])*0.6

ax= sns.lineplot(x=np.arange(0,b_data.shape[0]),y=b_data.Price,label='Actual',color='green')
ax= sns.lineplot(x=np.arange(0,b_data.shape[0]),y=ens_Predict,label='Stacked Prediciton',color='red')


In [ ]:
print('Stacked Model RMSE: ',np.sqrt(mean_squared_error(ens_Predict,b_data.Price)))

In [ ]:
ensmble_results = pd.DataFrame({'LR_Pred':LR_pipe.predict(b_data[features]),
                               'PR_Pred':PR_pipe.predict(b_data[features]),
                               'KNR_Pred':KNR_pipe.predict(b_data[features]),
                               'RF_Pred':RF_pipe.predict(b_data[features])})

ensmble_results.head(5)

In [ ]:

train_x,test_x,train_y,test_y = train_test_split(ensmble_results,b_data.Price,random_state=42)
xgb_m = XGBRegressor(random_state=42)


xgb_m.fit(train_x,train_y)
xgb_predictions = xgb_m.predict(test_x)

In [ ]:
ax= sns.lineplot(x=np.arange(0,test_y.shape[0]),y=test_y,label='Actual',color='green')
ax= sns.lineplot(x=np.arange(0,test_y.shape[0]),y=xgb_predictions,label='Stacked Prediciton',color='red')

In [ ]:
print('Ensemble Model RMSE: ',np.sqrt(mean_squared_error(test_y,xgb_predictions)))